## Noisy Simulations and Error Mitigation

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_aer import Aer

import numpy as np
from itertools import product

def create_calibration_matrix():
    """Builds a 4x4 measurement error calibration matrix for 2 qubits."""

    cal_circuits = []
    basis_states = [''.join(bits) for bits in product('01', repeat=2)]

    for state in basis_states:
        qc = QuantumCircuit(2, 2)
        for i, bit in enumerate(reversed(state)):  # Little endian
            if bit == '1':
                qc.x(i)
        qc.measure([0, 1], [0, 1])
        cal_circuits.append(qc)

    shots = 100000
    sim = FakeManilaV2()
    results = sim.run(transpile(cal_circuits, sim),shots=shots).result()
    counts = results.get_counts()

    M = np.zeros((4, 4))  # 4 prepared states x 4 measured states
    label_to_index = {label: idx for idx, label in enumerate(basis_states)}

    for i, state in enumerate(basis_states):
        counts = results.get_counts(cal_circuits[i])
        total = sum(counts.values())
        for bitstring, count in counts.items():
            # Ensure all bitstrings are 2 bits
            padded = bitstring.zfill(2)
            j = label_to_index[padded]
            M[j, i] = count / total
    return M, basis_states

def run_noisy_circuit():
    """Runs a 2-qubit Bell circuit that ideally outputs |00⟩ and |11⟩."""
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])

    sim = FakeManilaV2()
    shots = 10000
    result = sim.run(transpile(qc, sim),shots=shots).result()
    counts = result.get_counts()

    # Convert counts to probability vector
    p_meas = np.zeros(4)
    index_map = {'00': 0, '01': 1, '10': 2, '11': 3}
    for bitstring, count in counts.items():
        padded = bitstring.zfill(2)
        idx = index_map[padded]
        p_meas[idx] = count / shots

    return p_meas, counts

def apply_correction(M, p_meas):
    """Applies inverse of calibration matrix to correct measured probabilities."""
    M_inv = np.linalg.pinv(M)  # use pseudo-inverse for stability
    p_corrected = np.dot(M_inv, p_meas)
    p_corrected = np.clip(p_corrected, 0, 1)
    return p_corrected / np.sum(p_corrected)

In [ ]:
M, state_labels = create_calibration_matrix()
p_meas, raw_counts = run_noisy_circuit()
p_corrected = apply_correction(M, p_meas)

# Display
print("Calibration matrix M:")
print(np.round(M, 4))
print("\nMeasured probabilities:")
print(dict(zip(state_labels, np.round(p_meas, 4))))
print("\nCorrected probabilities:")
print(dict(zip(state_labels, np.round(p_corrected, 4))))

"""
    Plots a comparison between raw and corrected measurement probabilities.

    Args:
        state_labels (List[str]): List of bitstrings for basis states (e.g., ['00', '01', '10', '11']).
        p_meas (np.ndarray): Raw measured probabilities.
        p_corrected (np.ndarray): Corrected probabilities after error mitigation.
    """
raw_dict = dict(zip(state_labels, p_meas))
corrected_dict = dict(zip(state_labels, p_corrected))

plot_histogram(
    [raw_dict, corrected_dict],
    legend=['Raw', 'Corrected'],
    title='Measurement Error Mitigation',
    bar_labels=False
    )